# Exploratory Data Analysis (EDA) Report

- Author: [Kiruthikaa](https://github.com/kiruthikaa2512/)
- Repository: [datafun-05-sql-project](https://github.com/kiruthikaa2512/datafun-05-sql-project)
- Date: June 04, 2025
- Purpose: The purpose of this project is to explore and analyze a relational database that tracks technical issues and their corresponding analyses. Using Python and SQL, we aim to uncover patterns, trends, and insights from the structured data to better understand issue frequency, resolution status, and the types of analysis performed. This will demonstrate the integration of SQL with Python for effective data exploration and decision support.

## Introduction
### Dataset Overview:
The dataset contains two related tables — issues and analysis. The issues table records software/system issues with details like task ID, description, category, creation date, and status. The analysis table captures root cause analyses linked to issues via task IDs, including analysis type, root causes, resolutions, and resolution dates.

Goals:
Understand the distribution of issue types, their statuses, root causes, and resolution effectiveness to identify patterns and areas for process improvements.

# 1. Utils Logger Script

In [ ]:
"""
Logger Setup Script
File: utils_logger.py
"""
import logging
import pathlib 
import os
# Imports from external packages
from loguru import logger

# Setup a simple logger
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)




CURRENT_SCRIPT = pathlib.Path("__main__").stem  # Default to "__main__" if running as a script
print(CURRENT_SCRIPT)  # If running as a script, use the actual file name

# Set directory where logs will be stored
LOG_FOLDER: pathlib.Path = pathlib.Path("logs")

# Set the name of the log file
LOG_FILE: pathlib.Path = LOG_FOLDER.joinpath("project_log.log")

# Ensure the log folder exists or create it
try:
    LOG_FOLDER.mkdir(exist_ok=True)
    logger.info(f"Log folder created at: {LOG_FOLDER}")
except Exception as e:
    logger.error(f"Error creating log folder: {e}")

# Configure Loguru to write to the log file
try:
    logger.add(LOG_FILE, level="INFO")
    logger.info(f"Logging to file: {LOG_FILE}")
except Exception as e:
    logger.error(f"Error configuring logger to write to file: {e}")


def get_log_file_path() -> pathlib.Path:
    """Return the path to the log file."""
    return LOG_FILE


def log_example() -> None:
    """Example logging function to demonstrate logging behavior."""
    try:
        logger.info("This is an example info message.")
        logger.warning("This is an example warning message.")
        logger.error("This is an example error message.")
    except Exception as e:
        logger.error(f"An error occurred during logging: {e}")


def main() -> None:
    """Main function to execute logger setup and demonstrate its usage."""
    logger.info(f"STARTING {CURRENT_SCRIPT}.py")

    # Call the example logging function
    log_example()

    logger.info(f"View the log output at {LOG_FILE}")
    logger.info(f"EXITING {CURRENT_SCRIPT}.py.")


# Conditional execution block that calls main() only when this file is executed directly
if __name__ == "__main__":
    main()


2025-06-05 11:13:25.993 | INFO     | __main__:<module>:32 - Log folder created at: logs
2025-06-05 11:13:25.997 | INFO     | __main__:<module>:39 - Logging to file: logs\project_log.log
2025-06-05 11:13:25.999 | INFO     | __main__:main:61 - STARTING __main__.py
2025-06-05 11:13:25.999 | INFO     | __main__:log_example:52 - This is an example info message.
2025-06-05 11:13:26.000 | WARNING  | __main__:log_example:53 - This is an example warning message.
2025-06-05 11:13:26.000 | ERROR    | __main__:log_example:54 - This is an example error message.
2025-06-05 11:13:26.001 | INFO     | __main__:main:66 - View the log output at logs\project_log.log
2025-06-05 11:13:26.001 | INFO     | __main__:main:67 - EXITING __main__.py.


__main__


## Step 2: Database Setup
Creating the SQLite Database Using Python
Connect to the SQLite database stored in the data folder.
Use pandas and sqlite3 to run SQL queries and explore the results as DataFrames.

In [50]:
from pathlib import Path
import sys
import sqlite3
sys.path.append(str(Path('.').resolve()))

ROOT_DIR = Path.cwd().resolve()
DATA_FOLDER = ROOT_DIR / "data"
DB_PATH = Path(r"C:\Projects\Repos\datafun-05-sql-project\data\project.sqlite3")

# Ensure data folder exists
DATA_FOLDER.mkdir(exist_ok=True)

# Establish database connection
try:
    connection = sqlite3.connect(DB_PATH)
    logger.info(f"Connected to database: {DB_PATH}")
except Exception as e:
    logger.error(f"Connection error: {e}")

2025-06-05 22:19:49.004 | INFO     | __main__:<module>:16 - Connected to database: C:\Projects\Repos\datafun-05-sql-project\data\project.sqlite3


## 3. Creating Tables
We set up the schema with **CREATE TABLE** statements. This ensures a one-to-many relationship between issues and analysis.

In [52]:
import textwrap
import pandas as pd

cursor = connection.cursor()  # Explicitly define cursor

# Enable Foreign Key Support
cursor.execute("PRAGMA foreign_keys = ON;")
connection.commit()
print("Foreign keys enabled.")

# Drop existing tables

cursor.execute("DROP TABLE IF EXISTS analysis;")
cursor.execute("DROP TABLE IF EXISTS issues;")
connection.commit()
print("Tables dropped successfully.")
logger.info("Tables dropped successfully.")

# Create Issues Table
sql_create_issues = textwrap.dedent("""
    CREATE TABLE issues (
        task_id TEXT PRIMARY KEY,
        description TEXT,
        category TEXT,
        created_date TEXT,
        status TEXT
    );
""")

# Create Analysis Table
sql_create_analysis = textwrap.dedent("""
    CREATE TABLE analysis (
        analysis_id TEXT PRIMARY KEY,
        task_id TEXT,
        type_of_analysis TEXT,
        root_cause TEXT,
        resolution TEXT,
        resolution_date DATE,
        FOREIGN KEY (task_id) REFERENCES issues (task_id)
    );
""")

# Execute CREATE TABLE commands
cursor.execute(sql_create_issues)
cursor.execute(sql_create_analysis)
connection.commit()
print("Tables created successfully.")
logger.info("Database tables created successfully.")

# **Display created tables in the notebook**
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
df_tables = pd.DataFrame(tables, columns=["Table Name"])
print("Tables in database:")
display(df_tables)  # Ensures tables are shown clearly in Jupyter Notebook

2025-06-05 22:22:01.603 | INFO     | __main__:<module>:17 - Tables dropped successfully.


2025-06-05 22:22:01.621 | INFO     | __main__:<module>:48 - Database tables created successfully.


Foreign keys enabled.
Tables dropped successfully.
Tables created successfully.
Tables in database:


,Table Name
0,issues
1,analysis


   ## 4. Data Insertion
   We insert sample records into the database for both the tables that we created in our prrevious step.

In [ ]:
# Optional cleanup: remove existing records to avoid PRIMARY KEY conflicts
cursor.executescript("""
DELETE FROM analysis;
DELETE FROM issues;
""")
connection.commit()
# Sample data insertion script for `issues` and `analysis` tables
# This script inserts sample records into the `issues` and `analysis` tables.
# Insert records into `issues`
insert_issues_sql = textwrap.dedent(""" INSERT INTO issues (task_id, description, category, created_date, status) VALUES
('T23465', 'System outage in zone 1', 'Infrastructure', '2024-05-01', 'Closed'),
('T23468', 'Incorrect data on dashboard', 'Data Quality', '2024-05-03', 'Open'),
('T23475', 'Delay in daily ETL process', 'ETL', '2024-05-04', 'In Progress'),
('T23865', 'Frequent login failures', 'Authentication', '2024-05-06', 'Closed'),
('T23363', 'Missing entries in report', 'Reporting', '2024-05-07', 'Open'),
('T23432', 'Deployment failed to include several backend services causing runtime errors in production.', 'Deployment Error', '2025-05-16', 'Closed'),
('T23168', 'Critical bugs were found in the latest release that slipped through due to missing automated tests.', 'Quality Assurance', '2025-05-27', 'Closed'),
('T23482', 'Inconsistent system behavior across environments traced back to differences in config files', 'Configuration Management', '2025-05-28', 'Open'),
('T23861', 'Generated reports contained invalid values due to data corruption during nightly ETL processes.', 'Data Integrity', '2025-05-05', 'In Progress'),
('T23333', 'Feature development was misaligned with business needs due to lack of early stakeholder feedback.', 'Requirements Gathering', '2025-05-05', 'Closed');
""")

# View all records from 'issues'
df_issues = pd.read_sql_query("SELECT * FROM issues;", connection)
display(df_issues)
print(f"Issues dataframe shape: {df_issues.shape}")

# Insert records into `analysis`
insert_analysis_sql = textwrap.dedent("""
    INSERT INTO analysis (task_id, analysis_id, type_of_analysis, root_cause, resolution, resolution_date) VALUES
    ('T23465', 'A001', '5 Whys', 'Power supply issue', 'Installed UPS', '2025-05-02'),
('T23468', 'A002', 'Fishbone', 'Wrong join logic in query', 'Fixed join conditions', '2025-05-04'),
('T23475', 'A003', 'Pareto', 'Dependency on external API', 'Added retry mechanism', '2025-05-05'),
('T23865', 'A004', 'Root Cause Tree', 'Session timeout misconfig', 'Increased session timeout', '2025-05-07'),
('T23363', 'A005', '5 Whys', 'Scheduled job skipped', 'Rescheduled job', '2025-05-08'),
('T23432', 'A006', 'Fishbone', 'Incomplete deployment', 'Re-deployed with all components verified', '2025-05-17'),
('T23168', 'A007', 'Pareto', 'Lack of test coverage', 'Added unit and integration tests', '2025-05-29'),
('T23482', 'A008', 'Root Cause Tree', 'Configuration drift', 'Standardized and locked configuration', '2025-06-29'),
('T23861', 'A009', 'Root Cause Tree', 'Data corruption during ETL', 'Implemented checksums and validation', '2025-05-07'),
('T23333', 'A010', 'Root Cause Tree', 'Missing stakeholder input', 'Conducted stakeholder review session', '2025-05-07');
""")

# View all records from 'analysis'
df_analysis = pd.read_sql_query("SELECT * FROM analysis;", connection)
display(df_analysis)
print(f"Analysis dataframe shape: {df_analysis.shape}")

# Execute both inserts
cursor.executescript(insert_issues_sql + insert_analysis_sql)
connection.commit()

print("Sample records inserted successfully.")
logger.info("Sample data inserted into 'issues' and 'analysis' tables.")

# Expand display settings for full visibility
pd.set_option('display.max_rows', None)         # Show all rows
pd.set_option('display.max_columns', None)      # Show all columns
pd.set_option('display.max_colwidth', None)     # Do not truncate column content
pd.set_option('display.expand_frame_repr', False)  # Don't wrap DataFrame across lines

,task_id,description,category,created_date,status


Issues dataframe shape: (0, 5)


,analysis_id,task_id,type_of_analysis,root_cause,resolution,resolution_date


2025-06-05 22:22:19.794 | INFO     | __main__:<module>:53 - Sample data inserted into 'issues' and 'analysis' tables.


Analysis dataframe shape: (0, 6)
Sample records inserted successfully.


## **5. Querying the Data**
Read Records

In [79]:
# View all records from 'issues'
df_issues = pd.read_sql_query("SELECT * FROM issues;", connection)
display(df_issues)
print(f"Issues dataframe shape: {df_issues.shape}")

# View all records from 'analysis'
df_analysis = pd.read_sql_query("SELECT * FROM analysis;", connection)
display(df_analysis)
print(f"Analysis dataframe shape: {df_analysis.shape}")

for table in ['issues', 'analysis']:
    count_query = f"SELECT COUNT(*) FROM {table};"
    count = pd.read_sql_query(count_query, connection).iloc[0,0]
    print(f"Table '{table}' has {count} rows")

    logger.info(f"Table '{table}' has {count} rows")

,task_id,description,category,created_date,status
0,T23465,System outage in zone 1,Infrastructure,2024-05-01,Closed
1,T23468,Incorrect data on dashboard,Data Quality,2024-05-03,Resolved
2,T23475,Delay in daily ETL process,ETL,2024-05-04,In Progress
3,T23865,Frequent login failures,Authentication,2024-05-06,Closed
4,T23363,Missing entries in report,Reporting,2024-05-07,Open
5,T23432,Deployment failed to include several backend services causing runtime errors in production.,Deployment Error,2025-05-16,Closed
6,T23168,Critical bugs were found in the latest release that slipped through due to missing automated tests.,Quality Assurance,2025-05-27,Closed
7,T23482,Inconsistent system behavior across environments traced back to differences in config files,Configuration Management,2025-05-28,Open
8,T23861,Generated reports contained invalid values due to data corruption during nightly ETL processes.,Data Integrity,2025-05-05,In Progress
9,T23333,Feature development was misaligned with business needs due to lack of early stakeholder feedback.,Requirements Gathering,2025-05-05,Closed


Issues dataframe shape: (10, 5)


,analysis_id,task_id,type_of_analysis,root_cause,resolution,resolution_date
0,A001,T23465,5 Whys,Power supply issue,Installed UPS,2025-05-02
1,A002,T23468,Fishbone,Wrong join logic in query,Fixed join conditions,2025-05-04
2,A003,T23475,Pareto,Dependency on external API,Added retry mechanism,2025-05-05
3,A004,T23865,Root Cause Tree,Session timeout misconfig,Increased session timeout,2025-05-07
4,A005,T23363,5 Whys,Scheduled job skipped,Rescheduled job,2025-05-08
5,A006,T23432,Fishbone,Incomplete deployment,Re-deployed with all components verified,2025-05-17
6,A007,T23168,Pareto,Lack of test coverage,Added unit and integration tests,2025-05-29
7,A008,T23482,Root Cause Tree,Configuration drift,Standardized and locked configuration,2025-06-29
8,A009,T23861,Root Cause Tree,Data corruption during ETL,Implemented checksums and validation,2025-05-07
9,A010,T23333,Root Cause Tree,Missing stakeholder input,Conducted stakeholder review session,2025-05-07


2025-06-05 22:37:01.985 | INFO     | __main__:<module>:16 - Table 'issues' has 10 rows
2025-06-05 22:37:01.989 | INFO     | __main__:<module>:16 - Table 'analysis' has 10 rows


Analysis dataframe shape: (10, 6)
Table 'issues' has 10 rows
Table 'analysis' has 10 rows


## 6. Basic SQL Queries
### Filtered Views

In [80]:

#Closed issues
pd.read_sql_query("SELECT * FROM issues WHERE status = 'Closed';", connection)
display(df_issues[df_issues['status'] == 'Closed'])
logger.info("Displayed closed issues.")


,task_id,description,category,created_date,status
0,T23465,System outage in zone 1,Infrastructure,2024-05-01,Closed
3,T23865,Frequent login failures,Authentication,2024-05-06,Closed
5,T23432,Deployment failed to include several backend services causing runtime errors in production.,Deployment Error,2025-05-16,Closed
6,T23168,Critical bugs were found in the latest release that slipped through due to missing automated tests.,Quality Assurance,2025-05-27,Closed
9,T23333,Feature development was misaligned with business needs due to lack of early stakeholder feedback.,Requirements Gathering,2025-05-05,Closed


2025-06-05 22:37:02.034 | INFO     | __main__:<module>:4 - Displayed closed issues.


In [81]:
# Unique categories
pd.read_sql_query("SELECT DISTINCT category FROM issues;", connection)
display(pd.read_sql_query("SELECT DISTINCT category FROM issues;", connection))
print("Unique categories in 'issues':")
logger.info("Unique categories in 'issues':")

,category
0,Infrastructure
1,Data Quality
2,ETL
3,Authentication
4,Reporting
5,Deployment Error
6,Quality Assurance
7,Configuration Management
8,Data Integrity
9,Requirements Gathering


2025-06-05 22:37:02.058 | INFO     | __main__:<module>:5 - Unique categories in 'issues':


Unique categories in 'issues':


In [82]:
#Ordered Records
pd.read_sql_query("SELECT * FROM issues ORDER BY created_date DESC;", connection)
display(pd.read_sql_query("SELECT * FROM issues ORDER BY created_date DESC;", connection))
print("Issues ordered by created date in descending order.")
logger.info("Issues ordered by created date in descending order.")

,task_id,description,category,created_date,status
0,T23482,Inconsistent system behavior across environments traced back to differences in config files,Configuration Management,2025-05-28,Open
1,T23168,Critical bugs were found in the latest release that slipped through due to missing automated tests.,Quality Assurance,2025-05-27,Closed
2,T23432,Deployment failed to include several backend services causing runtime errors in production.,Deployment Error,2025-05-16,Closed
3,T23861,Generated reports contained invalid values due to data corruption during nightly ETL processes.,Data Integrity,2025-05-05,In Progress
4,T23333,Feature development was misaligned with business needs due to lack of early stakeholder feedback.,Requirements Gathering,2025-05-05,Closed
5,T23363,Missing entries in report,Reporting,2024-05-07,Open
6,T23865,Frequent login failures,Authentication,2024-05-06,Closed
7,T23475,Delay in daily ETL process,ETL,2024-05-04,In Progress
8,T23468,Incorrect data on dashboard,Data Quality,2024-05-03,Resolved
9,T23465,System outage in zone 1,Infrastructure,2024-05-01,Closed


2025-06-05 22:37:02.092 | INFO     | __main__:<module>:5 - Issues ordered by created date in descending order.


Issues ordered by created date in descending order.


## 7. Joining Tables

### Analyze Issue Root Causes
It retrieves each issue’s ID and description along with its root cause and resolution by joining the issues and analysis tables on task_id.


In [83]:
query = """
SELECT i.task_id, i.description, a.root_cause, a.resolution
FROM issues i
JOIN analysis a ON i.task_id = a.task_id;
"""
pd.read_sql_query(query, connection)
print("Query executed successfully.")
display(pd.read_sql_query(query, connection))
logger.info("Query executed successfully and results displayed.")

Query executed successfully.


,task_id,description,root_cause,resolution
0,T23465,System outage in zone 1,Power supply issue,Installed UPS
1,T23468,Incorrect data on dashboard,Wrong join logic in query,Fixed join conditions
2,T23475,Delay in daily ETL process,Dependency on external API,Added retry mechanism
3,T23865,Frequent login failures,Session timeout misconfig,Increased session timeout
4,T23363,Missing entries in report,Scheduled job skipped,Rescheduled job
5,T23432,Deployment failed to include several backend services causing runtime errors in production.,Incomplete deployment,Re-deployed with all components verified
6,T23168,Critical bugs were found in the latest release that slipped through due to missing automated tests.,Lack of test coverage,Added unit and integration tests
7,T23482,Inconsistent system behavior across environments traced back to differences in config files,Configuration drift,Standardized and locked configuration
8,T23861,Generated reports contained invalid values due to data corruption during nightly ETL processes.,Data corruption during ETL,Implemented checksums and validation
9,T23333,Feature development was misaligned with business needs due to lack of early stakeholder feedback.,Missing stakeholder input,Conducted stakeholder review session


2025-06-05 22:37:02.132 | INFO     | __main__:<module>:9 - Query executed successfully and results displayed.


## 8. Updates & Deletions
It updates the status to Resolved for the task id given in the query.

In [84]:
connection.execute("UPDATE issues SET status = 'Resolved' WHERE task_id = 'T23468';")
connection.commit()
print("Updated status of task T23468 to 'Resolved'.")
display(pd.read_sql_query("SELECT * FROM issues WHERE task_id = 'T23468';", connection))
logger.info("Updated status of task T23468 to 'Resolved'.")

Updated status of task T23468 to 'Resolved'.


,task_id,description,category,created_date,status
0,T23468,Incorrect data on dashboard,Data Quality,2024-05-03,Resolved


2025-06-05 22:37:02.172 | INFO     | __main__:<module>:5 - Updated status of task T23468 to 'Resolved'.


## 9. Aggregations & Grouping

### Issue Count by Category: Displays the count per each category

In [85]:
query = "SELECT category, COUNT(*) as count FROM issues GROUP BY category;"
df_category_count = pd.read_sql_query(query, connection)
df_category_count
print("Category counts:")
display(df_category_count)
logger.info("Category counts displayed.")

Category counts:


,category,count
0,Authentication,1
1,Configuration Management,1
2,Data Integrity,1
3,Data Quality,1
4,Deployment Error,1
5,ETL,1
6,Infrastructure,1
7,Quality Assurance,1
8,Reporting,1
9,Requirements Gathering,1


2025-06-05 22:38:22.011 | INFO     | __main__:<module>:6 - Category counts displayed.
